In [1]:
import allegro.model as al
from nequip.model import model_from_config
from nequip.data import AtomicData
import yaml
import torch
from torch_geometric.datasets import QM9

In [ ]:
# Modifications needed in the nequip install
# 1) Install pytorch geometric

# 2) nequip/model/_scaling.py
# --> comment out lines 257-259

# 3) nequip/data/AtomicData.py
# --> add at the beginning:
# from torch_geometric.data.data import Data as geom_Data
# --> add the following after ~ line 571:
# elif isinstance(data, geom_Data):
#     keys = data.keys

In [2]:
torch.__version__

'1.11.0+cu102'

In [3]:
# Put your config path to an allegro config file
config_path = '/home/tin/Documents/GitHub/Allegro/allegro/configs/minimal.yaml'
with open(config_path, "r") as stream:
    try:
        config  = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
# Needed to intialize a model from config, needs config, initialize, and an optional dataset
# The dataset is needed to calculate the average number of neigbors
# For this, put something like avg_num_neighbors: 5 in the config to avoid this issue
initialize = False
model = model_from_config(config, initialize)

builder: <class 'nequip.data.transforms.TypeMapper'>
builder type: <class 'type'>
builder: <class 'nequip.nn.embedding._one_hot.OneHotAtomEncoding'>
builder type: <class 'type'>
builder: <class 'nequip.nn.embedding._edge.RadialBasisEdgeEncoding'>
builder type: <class 'type'>
builder: <class 'allegro.nn._norm_basis.NormalizedBasis'>
builder type: <class 'type'>
builder: <class 'nequip.nn.radial_basis.BesselBasis'>
builder type: <class 'type'>
builder: <class 'nequip.nn.cutoffs.PolynomialCutoff'>
builder type: <class 'type'>
builder: <class 'nequip.nn.embedding._edge.SphericalHarmonicEdgeAttrs'>
builder type: <class 'type'>
builder: <class 'allegro.nn._allegro.Allegro_Module'>
builder type: <class 'type'>
builder: <class 'allegro.nn._fc.ScalarMLPFunction'>
builder type: <class 'type'>
builder: <class 'allegro.nn._fc.ScalarMLPFunction'>
builder type: <class 'type'>
builder: <class 'allegro.nn._fc.ScalarMLPFunction'>
builder type: <class 'type'>
builder: <class 'allegro.nn._fc.ScalarMLP'>


In [1]:
at_data.neighbors

NameError: name 'at_data' is not defined

In [6]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

model.func.radial_basis.basis.basis.bessel_weights
model.func.allegro.latents.0._forward._weight_0
model.func.allegro.latents.0._forward._weight_1
model.func.allegro.env_embed_mlps.0._forward._weight_0
model.func.allegro.linears.0.w
model.func.allegro.final_latent._forward._weight_0
model.func.edge_eng._module._forward._weight_0
model.func.edge_eng._module._forward._weight_1


In [7]:
# Import QM9, take a data sample to test whether we can do a forward pass
dataset = QM9('./')
data = dataset[0]

In [12]:
# Imitating the data forward pass
data = AtomicData.to_AtomicDataDict(data)

data_unscaled = data
# This block is also an issue, so I have commented it out
'''for layer in self.rescale_layers:
    # This means that self.model is RescaleOutputs
    # this will normalize the targets
    # in validation (eval mode), it does nothing
    # in train mode, if normalizes the targets
    data_unscaled = layer.unscale(data_unscaled)

# Run model
# We make a shallow copy of the input dict in case the model modifies it
input_data = {
    k: v
    for k, v in data_unscaled.items()
    if k not in self._remove_from_model_input
}'''
input_data = data # This is added because there is a commented out portion of the code
out = model(input_data)

# Good luck have fun babes

KeyError: 'atom_types'